<a href="https://colab.research.google.com/github/julianl11/projektarbeit_1/blob/main/03_computer_vision_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start GO!

In [ ]:
"hello"

In [ ]:
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

In [ ]:
train_data = datasets.FashionMNIST(
    root="data", # where to download data to?
    train=True, # get training data
    download=True, # download data if it doesn't exist on disk
    transform=ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # you can transform labels as well
)

# Setup testing data
test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)

In [ ]:
len(train_data[0])

In [ ]:
img, label = train_data[0]
print(img.shape, label)

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
import matplotlib.pyplot as plt
image, label = train_data[0]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze()) # image shape is [1, 28, 28] (colour channels, height, width)
plt.title(label);

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
(next(iter(train_dataloader))[0]).shape

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis("Off");
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

In [ ]:
flatten_model = nn.Flatten()

x = train_features_batch[0]

print(f"Shape before flattening: {x.shape}")
print(f"Shape after flattening: {flatten_model(x).shape}")
print(f"Dim after flattening: {flatten_model(x).dim()}")
#flatten_model(x)

In [ ]:
from torch import nn

class FashionMNISTModelV1(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.Linear(in_features=hidden_units, out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    return self.layer_stack(x)

In [ ]:
int(class_names.__len__())

In [ ]:
model_1 = FashionMNISTModelV1(input_shape=784, hidden_units=10, output_shape=int(class_names.__len__()))
model_1.to("cpu")

In [ ]:
dummy_x = torch.randn((1, 28, 28))
dummy_x.shape

In [ ]:
model_1(dummy_x)

In [ ]:
model_1(dummy_x).squeeze().__len__()

In [ ]:
model_1.state_dict()

In [ ]:
model_1.state_dict()["layer_stack.1.weight"].squeeze()[0].__len__()

In [ ]:
import requests
from pathlib import Path

if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_1.parameters(), lr = 0.01)

In [ ]:
next(model_1.parameters()).squeeze().shape

In [ ]:
from timeit import default_timer as timer

def print_train_time(start: float, end:float, device: torch.device = None):
  total_time = end-start
  print(f"Train tim on: {device} took {total_time:.3f} seconds")
  return total_time

In [ ]:
start_time = timer()

end_time = timer()
print_train_time(start_time, end_time, device="cpu")

## optimizer will step per batch
## loss will be calculated per batch

In [ ]:
X, y  = next(iter(train_dataloader))
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}, {y}")

In [ ]:
dummy_x.shape

In [ ]:
model_1(dummy_x)

In [ ]:
y = torch.tensor([1])
print(f"{model_1(dummy_x)}, {y}")
loss_fn(model_1(dummy_x), y)

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}")

  train_loss = 0

  for batch, (X, y) in enumerate(train_dataloader):
    model_1.train()
    y_pred = model_1(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if batch % 400 == 0:
    print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss = train_loss / len(train_dataloader)
  print(f"Train loss: {train_loss}")

  test_loss, test_acc = 0, 0

  model_1.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      test_pred = model_1(X_test)
      test_loss += loss_fn(test_pred, y_test)
      test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

  test_loss = test_loss / len(test_dataloader)
  test_acc = test_acc / len(test_dataloader)

  print(f"Test loss: {test_loss} | Test accuracy: {test_acc}")

train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu, end=train_time_end_on_cpu, device=str(next(model_1.parameters()).device))


In [ ]:
X, y = next(iter(test_dataloader))
X, y.__len__()

In [ ]:
def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn):
  """Return dictionary containing the results of model predicting on data_loader"""
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, y in data_loader:
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      loss = loss + loss_fn(y_pred, y)
      acc = acc + accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
  loss = loss / len(data_loader)
  acc = acc / len(data_loader)

  return {"model_name": model.__class__.__name__,
          "model_loss": loss.item(),
          "model_acc": acc}

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
eval_model(model_1, test_dataloader, loss_fn, accuracy_fn)

In [ ]:
model_1.__class__.__name__

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class FashionMNISTModelV2(nn.Module):

  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_shape),
        nn.ReLU()
    )

  def forward(self, x: torch.Tensor):
    return self.layer_stack(x)

In [ ]:
model_2 = FashionMNISTModelV2(input_shape=784, hidden_units=10, output_shape=10)
model_2.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_2.parameters(), lr=0.01)

In [ ]:
loss_fn, optimizer

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}")

  train_loss = 0

  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    model_2.train()
    y_pred = model_2(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if batch % 400 == 0:
    print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss = train_loss / len(train_dataloader)
  print(f"Train loss: {train_loss}")

  test_loss, test_acc = 0, 0

  model_2.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      X_test, y_test = X_test.to(device), y_test.to(device)
      test_pred = model_2(X_test)
      test_loss += loss_fn(test_pred, y_test)
      test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

  test_loss = test_loss / len(test_dataloader)
  test_acc = test_acc / len(test_dataloader)

  print(f"Test loss: {test_loss} | Test accuracy: {test_acc}")

train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu, end=train_time_end_on_cpu, device=str(next(model_2.parameters()).device))


In [ ]:
eval_model(model_2, test_dataloader, loss_fn, accuracy_fn)

In [ ]:
74.52076677316293 == eval_model(model_2, test_dataloader, loss_fn, accuracy_fn)["model_acc"]

In [ ]:
def train_func(model: torch.nn.Module,
               epochs: int,
               train_dataloader: torch.utils.data.DataLoader,
               test_dataloader: torch.utils.data.DataLoader,
               device: torch.device):
  model.to(device)

  for epoch in range(epochs):
    print(f"Epoch: {epoch}")

    train_loss = 0

    for batch, (X, y) in enumerate(train_dataloader):
      model.train()

      X, y = X.to(device), y.to(device)

      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      train_loss += loss
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    train_loss = train_loss / len(train_dataloader)
    print(f"Train loss: {train_loss}")



In [ ]:
train_func(model_2, 3, train_dataloader, test_dataloader, device)

In [ ]:
def test_func(model: torch.nn.Module,
              train_dataloader: torch.utils.data.DataLoader,
              test_dataloader: torch.utils.data.DataLoader,
              device: torch.device,
              accuracy_fn ):


    test_loss, test_acc = 0, 0

    model.eval()
    with torch.inference_mode():
      for X_test, y_test in test_dataloader:
        X_test, y_test = X_test.to(device), y_test.to(device)
        test_pred = model(X_test)
        test_loss += loss_fn(test_pred, y_test)
        test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    test_loss = test_loss / len(test_dataloader)
    test_acc = test_acc / len(test_dataloader)

    print(f"Test loss: {test_loss} | Test accuracy: {test_acc}")

In [ ]:
test_func(model_2, train_dataloader, test_dataloader, device, accuracy_fn)

In [ ]:
eval_model(model_2, test_dataloader, loss_fn, accuracy_fn)

# Creating CCN Model - Juhu!

In [ ]:
class FashionMNISTModelV3(nn.Module):

  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()

    self.conv1_block = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=(3,3), stride=1, padding=1), # shape 28*28
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=(3,3), stride=1, padding=1), # 26*26
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2)) # 12 * 12
    )

    self.conv2_block = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=(3,3), stride=1, padding=1), # 11*11
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=(3,3), stride=1, padding=1), # 9 * 9
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2)) # 4 * 4
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*7*7, out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv1_block(x)
    x = self.conv2_block(x)
    x = self.classifier(x)
    return x

In [ ]:
torch.tensor([49]).to(device).item()

In [ ]:
model_3 = FashionMNISTModelV3(input_shape=1, hidden_units=10, output_shape=10)
model_3

In [ ]:
torch.manual_seed(42)

images = torch.rand(size=(32, 1, 28, 28))
test_image = images[0]
print(test_image.shape)


In [ ]:
conv_layer = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3,3), stride=1, padding=10)
conv_layer

In [ ]:
conv_layer(test_image).shape

In [ ]:
conv_output = conv_layer(test_image)
conv_output

In [ ]:
image.shape

In [ ]:
image.unsqueeze(dim=1).shape

In [ ]:
pooling_layer = nn.MaxPool2d(kernel_size=(2))
pooling_layer

In [ ]:
pooling_layer(image).shape

In [ ]:
t = torch.randn([1,3,3])

In [ ]:
t.shape

In [ ]:
t.dim()

In [ ]:
t

In [ ]:
pooling_layer(t)

In [ ]:
t.max()

In [ ]:
plt.imshow(image.squeeze(), cmap="grey")

In [ ]:
device

In [ ]:
model_3.to(device)

In [ ]:
image = image.to(device)
image.device

In [ ]:
model_3.to(device)
model_3(image.unsqueeze(dim=0))

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_3.parameters(), lr=0.01)

# Testing and Training the CNN Model

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

train_time_start_on_cpu = timer()

for epoch in tqdm(range(3)):
  print(f"Epoch: {epoch}")
  train_func(model_3, 1, train_dataloader, test_dataloader, device)
  test_func(model_3, train_dataloader, test_dataloader, device, accuracy_fn)

train_time_end_on_cpu = timer()

print(f"Total train time: {print_train_time(start=train_time_start_on_cpu, end=train_time_end_on_cpu, device=str(next(model_3.parameters()).device))}")

In [ ]:
eval_model(model_3, test_dataloader, loss_fn, accuracy_fn)

In [ ]:
import pandas as pd

In [ ]:
results = pd.DataFrame([eval_model(model_3, test_dataloader, loss_fn, accuracy_fn), eval_model(model_2, test_dataloader, loss_fn, accuracy_fn)])
results

In [ ]:
results.set_index("model_name")["model_acc"].plot(kind="bar")

In [ ]:
plt.imshow(image.squeeze(), cmap="grey")

In [ ]:
model_3

In [ ]:
image.unsqueeze(dim=0).shape

In [ ]:
class_names[torch.argmax(model_3(image.unsqueeze(dim=0)))]

In [ ]:
class_names

In [ ]:
def make_predictions(model: torch.nn.Module, data: list, device: torch.device):
  pred_probs = []
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data:
      sample = sample.to(device)
      pred_logit = model(sample)
      pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)
      pred_probs.append(pred_prob.cpu())


  return torch.stack(pred_probs)



In [ ]:
list(test_dataloader)[0][1].__len__()

In [ ]:
import random
random.seed(42)
test_samples = []
test_lables = []
for sample, label in random.sample(list(test_dataloader), k=10):
  test_samples.append(sample)
  test_lables.append(label)

In [ ]:
torch.argmax(torch.softmax((make_predictions(model_3, test_samples, device))[0], dim=1), dim=1)

In [ ]:
test_lables[0]

In [ ]:
result_bool = torch.eq(test_lables[0], torch.argmax(torch.softmax((make_predictions(model_3, test_samples, device))[0], dim=1), dim=1))

In [ ]:
result_count = torch.sum(result_bool == True)

In [ ]:
f"{(result_count / 32)}%"

In [ ]:
test_samples[0][:10].shape

In [ ]:
plt.figure(figsize=(10,10))
nrows = 3
ncols = 3

for i, sample in enumerate(test_samples[0][:9]):
  plt.subplot(nrows, ncols, i+1)
  plt.imshow(sample.squeeze(), cmap="grey")
  pred_label = class_names[i]
  true_label = class_names[i]
  plt.title(f"Pred: {pred_label} | True: {true_label}")
  plt.axis(False)


In [ ]:
import mlxtend
print(mlxtend.__version__)

## matplot only works on cpu - data to cpu!!

In [ ]:
y_preds =[]
model_3.eval()

with torch.inference_mode():
  for x, y in tqdm(test_dataloader, desc="Making predictions..."):
    x, y = x.to(device), y.to(device)

    y_logits = model_3(x)
    y_pred = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
    y_preds.append(y_pred.cpu())

#print(y_preds)
y_pred_tensor = torch.cat(y_preds)
y_pred_tensor.shape

In [ ]:
#!pip install -q torchmetrics

In [ ]:
import mlxtend
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
confmat = mlxtend.evaluate.confusion_matrix(y_target=test_lables[0], y_predicted=y_pred_tensor)

In [ ]:
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "03_pytorch_cnn_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_3.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

In [ ]:
torch.manual_seed(42)

loaded_cnn_model = FashionMNISTModelV3(input_shape=1, hidden_units=10, output_shape=10)
loaded_cnn_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
loaded_cnn_model

In [ ]:
class_names[torch.argmax(loaded_cnn_model(image.unsqueeze(dim=0)))]

In [ ]:
plt.imshow(image.squeeze(), cmap="grey")

In [ ]:
loaded_model_results = eval_model(loaded_cnn_model, test_dataloader, loss_fn, accuracy_fn)
loaded_model_results

In [ ]:
model_results = eval_model(model_3, test_dataloader, loss_fn, accuracy_fn)
model_results

In [ ]:
torch.isclose(torch.tensor(model_results["model_acc"]), torch.tensor(loaded_model_results["model_acc"]))